# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [12]:
# import libraries

import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql("SELECT * FROM Messages", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [3]:
        def tokenize(text):
            """
            input:
            message

            Returning:
            list of words into numbers of same meaning
            """
            # Converting everything to lower case
            text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

            # Tokenize words
            tokens = word_tokenize(text)

            # normalization word tokens and remove stop words
            normalize = PorterStemmer()
            stop_words = stopwords.words("english")

            normalized = [normalize.stem(word) for word in tokens if word not in stop_words]

            return normalized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
#split data
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def eval_metrics(ArrayL, ArrayP, col_names):
    """Evalute metrics of the ML pipeline model
    
    inputs:
    ArrayL: array. Array containing the real labels.
    ArrayP: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the ArrayP fields.
       
    Returns:
    data_metrics: Contains accuracy, precision, recall 
    and f1 score for a given set of ArrayL and ArrayP labels.
    """
    metrics = []
    
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(ArrayL[:, i], ArrayP[:, i])
        precision = precision_score(ArrayL[:, i], ArrayP[:, i])
        recall = recall_score(ArrayL[:, i], ArrayP[:, i])
        f1 = f1_score(ArrayL[:, i], ArrayP[:, i])
        
        metrics.append([accuracy, precision, recall, f1])
    
    # store metrics
    metrics = np.array(metrics)
    data_metrics = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return data_metrics    

In [7]:
# Calculate evaluation metrics for training set
Y_train_pred = pipeline.predict(X_train)
col_names = list(Y.columns.values)

print(eval_metrics(np.array(y_train), Y_train_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.983532   0.982047  0.996584  0.989262
request                 0.975834   0.990866  0.867595  0.925142
offer                   0.998419   0.983607  0.666667  0.794702
aid_related             0.969766   0.991407  0.935274  0.962523
medical_help            0.979402   0.993186  0.746479  0.852339
medical_products        0.984960   0.992733  0.701953  0.822396
search_and_rescue       0.990976   1.000000  0.683929  0.812301
security                0.994953   1.000000  0.725000  0.840580
military                0.991231   1.000000  0.734568  0.846975
child_alone             1.000000   0.000000  0.000000  0.000000
water                   0.983481   0.992325  0.740589  0.848172
food                    0.975987   0.995319  0.786044  0.878389
shelter                 0.978026   0.993289  0.759407  0.860743
clothing                0.995615   0.991071  0.725490  0.837736
money                   0.994086   0.993

In [8]:
# Calculate evaluation metrics for test set
Y_test_pred = pipeline.predict(X_test)

eval_metrics_test = eval_metrics(np.array(y_test), Y_test_pred, col_names)
print(eval_metrics_test)



                        Accuracy  Precision    Recall        F1
related                 0.712035   0.760260  0.902375  0.825244
request                 0.822603   0.354167  0.077982  0.127820
offer                   0.995412   0.000000  0.000000  0.000000
aid_related             0.558036   0.430007  0.222263  0.293053
medical_help            0.914666   0.069767  0.005758  0.010638
medical_products        0.947698   0.000000  0.000000  0.000000
search_and_rescue       0.971861   0.250000  0.011111  0.021277
security                0.983178   0.000000  0.000000  0.000000
military                0.963450   0.000000  0.000000  0.000000
child_alone             1.000000   0.000000  0.000000  0.000000
water                   0.931641   0.031250  0.002398  0.004454
food                    0.882398   0.187500  0.020862  0.037547
shelter                 0.903961   0.081633  0.006814  0.012579
clothing                0.983790   0.000000  0.000000  0.000000
money                   0.977519   0.000

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
# Define performance metric for use in grid search scoring object
def perform_metric(Y_T, Y_P):
    """Median F1 score for all classifiers
    
    inputs:
    Y_T: array. Array containing ArrayL labels.
    Y_P: array. Array containing ArrayP labels.
        
    Routputs:
    Median F1 score for all  classifiers
    """
    f1_list = []
    for i in range(np.shape(Y_P)[1]):
        f1 = f1_score(np.array(Y_T)[:, i], Y_P[:, i])
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score



In [19]:
# grid search

parameters = {'vect__min_df': [1, 3],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[12, 14], 
              'clf__estimator__min_samples_split':[6]}

scorer = make_scorer(perform_metric)
cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer, verbose = 10)

# Find best parameters
np.random.seed(74)
T_model = cv.fit(X_train, y_train)



Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__min_samples_split=6, clf__estimator__n_estimators=12, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=6, clf__estimator__n_estimators=12, tfidf__use_idf=True, vect__min_df=1, score=0.008715662825251866, total= 1.2min
[CV] clf__estimator__min_samples_split=6, clf__estimator__n_estimators=12, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=6, clf__estimator__n_estimators=12, tfidf__use_idf=True, vect__min_df=1, score=0.0029673590504451035, total= 1.2min
[CV] clf__estimator__min_samples_split=6, clf__estimator__n_estimators=12, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=6, clf__estimator__n_estimators=12, tfidf__use_idf=True, vect__min_df=1, score=0.008108108108108109, total= 1.2min
[CV] clf__estimator__min_samples_split=6, clf__estimator__n_estimators=12, tfidf__use_idf=True, vect__min_df=3 


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 12.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=6, clf__estimator__n_estimators=12, tfidf__use_idf=False, vect__min_df=1, score=0.009684437062855272, total= 1.2min
[CV] clf__estimator__min_samples_split=6, clf__estimator__n_estimators=12, tfidf__use_idf=False, vect__min_df=3 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 13.5min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=6, clf__estimator__n_estimators=12, tfidf__use_idf=False, vect__min_df=3, score=0.00781875453599994, total= 1.1min
[CV] clf__estimator__min_samples_split=6, clf__estimator__n_estimators=12, tfidf__use_idf=False, vect__min_df=3 
[CV]  clf__estimator__min_samples_split=6, clf__estimator__n_estimators=12, tfidf__use_idf=False, vect__min_df=3, score=0.010775768299226987, total= 1.1min
[CV] clf__estimator__min_samples_split=6, clf__estimator__n_estimators=12, tfidf__use_idf=False, vect__min_df=3 
[CV]  clf__estimator__min_samples_split=6, clf__estimator__n_estimators=12, tfidf__use_idf=False, vect__min_df=3, score=0.011510315458717491, total= 1.1min
[CV] clf__estimator__min_samples_split=6, clf__estimator__n_estimators=14, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=6, clf__estimator__n_estimators=14, tfidf__use_idf=True, vect__min_df=1, score=0.009489930317119023, total= 1.3min
[CV] clf__estimator__min_samples_split=6

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 37.1min finished


In [20]:
#grid search results
T_model.cv_results_

{'mean_fit_time': array([ 60.51885684,  57.8440481 ,  59.76982848,  55.0184509 ,
         67.70449011,  65.15158359,  67.44481738,  61.39497828]),
 'std_fit_time': array([ 0.25335554,  0.68522454,  0.16463509,  0.31004568,  0.22116516,
         0.43850717,  0.48501612,  0.53449203]),
 'mean_score_time': array([ 10.27765028,   9.89492496,  10.18499796,   9.81885568,
         10.83773311,  10.33359281,  10.78072095,  10.1548423 ]),
 'std_score_time': array([ 0.12937303,  0.03047195,  0.04424241,  0.068262  ,  0.09308761,
         0.07219223,  0.03899409,  0.08299798]),
 'param_clf__estimator__min_samples_split': masked_array(data = [6 6 6 6 6 6 6 6],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_clf__estimator__n_estimators': masked_array(data = [12 12 12 12 14 14 14 14],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_tfidf__use_idf': masked_array(data = [True True False F

In [21]:
# top mean score
np.max(T_model.cv_results_['mean_test_score'])



0.011447389139491686

In [22]:
# Parameters for top mean score
T_model.best_params_

{'clf__estimator__min_samples_split': 6,
 'clf__estimator__n_estimators': 14,
 'tfidf__use_idf': False,
 'vect__min_df': 3}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
# evaluating metrics for test set
tuned_pred_test_1 = T_model.predict(X_test)

eval_metrics_test_1 = eval_metrics(np.array(y_test), tuned_pred_test_1, col_names)

print(eval_metrics_test_1)

                        Accuracy  Precision    Recall        F1
related                 0.737116   0.767000  0.940337  0.844870
request                 0.814039   0.406639  0.083689  0.138810
offer                   0.996024   0.000000  0.000000  0.000000
aid_related             0.549319   0.433550  0.269655  0.332503
medical_help            0.913289   0.092308  0.011673  0.020725
medical_products        0.946628   0.057143  0.006289  0.011331
search_and_rescue       0.969720   0.040000  0.005714  0.010000
security                0.980425   0.000000  0.000000  0.000000
military                0.963603   0.045455  0.004587  0.008333
child_alone             1.000000   0.000000  0.000000  0.000000
water                   0.927206   0.075000  0.006787  0.012448
food                    0.877198   0.178082  0.017196  0.031363
shelter                 0.899526   0.112676  0.013289  0.023774
clothing                0.983331   0.142857  0.009615  0.018018
money                   0.976143   0.000

In [25]:
# summary of 1st model
eval_metrics_test_1.describe()



,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.920808,0.137587,0.053943,0.065395
std,0.095037,0.195320,0.163938,0.158321
min,0.549319,0.000000,0.000000,0.000000
25%,0.907669,0.000000,0.000000,0.000000
50%,0.949304,0.059821,0.006121,0.011101
75%,0.981152,0.181181,0.015013,0.028111
max,1.000000,0.767000,0.940337,0.844870


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [33]:
# testing a pure decision tree classifier
moc = MultiOutputClassifier(DecisionTreeClassifier())

pipeline_3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', moc)
    ])

parameters_2 = {'vect__min_df': [1],
              'tfidf__use_idf':[True],
              'clf__estimator__criterion': ['gini'], 
                }

cv2 = GridSearchCV(pipeline_3, param_grid = parameters_2, scoring = scorer, verbose = 10)

# Find best parameters
np.random.seed(25)
T_model2 = cv2.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] clf__estimator__criterion=gini, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__criterion=gini, tfidf__use_idf=True, vect__min_df=1, score=0.05128205128205128, total= 6.0min
[CV] clf__estimator__criterion=gini, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.3min remaining:    0.0s


[CV]  clf__estimator__criterion=gini, tfidf__use_idf=True, vect__min_df=1, score=0.05402424159184163, total= 6.1min
[CV] clf__estimator__criterion=gini, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 12.6min remaining:    0.0s


[CV]  clf__estimator__criterion=gini, tfidf__use_idf=True, vect__min_df=1, score=0.049869109947643986, total= 6.2min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 19.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 19.1min finished


In [30]:
pipeline_3.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f5770a61620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, presort=False, random_state=None,
               splitter='best'),
              n_jobs=1))],
 

In [35]:
# Calculate evaluation metrics for test set
tuned_pred_test_2 = T_model2.predict(X_test)

eval_metrics_test_2 = eval_metrics(np.array(y_test), tuned_pred_test_2, col_names)

print(eval_metrics_test_2)

                        Accuracy  Precision    Recall        F1
related                 0.664475   0.768208  0.800924  0.784225
request                 0.776265   0.344350  0.275833  0.306306
offer                   0.993118   0.000000  0.000000  0.000000
aid_related             0.528368   0.433309  0.432035  0.432671
medical_help            0.867870   0.083333  0.068093  0.074946
medical_products        0.917419   0.048780  0.037736  0.042553
search_and_rescue       0.949686   0.047059  0.045714  0.046377
security                0.967732   0.020619  0.016949  0.018605
military                0.941275   0.048913  0.041284  0.044776
child_alone             1.000000   0.000000  0.000000  0.000000
water                   0.892338   0.110119  0.083710  0.095116
food                    0.822144   0.138544  0.103175  0.118271
shelter                 0.845236   0.091633  0.076412  0.083333
clothing                0.972932   0.060241  0.048077  0.053476
money                   0.959933   0.034

### 9. Export your model as a pickle file

In [26]:
# Pickle best model
pickle.dump(T_model, open('disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.